# Batch Mode on rowstore
Batch mode on rowstore enables batch mode execution for analytic workloads without requiring columnstore indexes. This feature supports batch mode execution and bitmap filters for on-disk heaps and B-tree indexes. Batch mode on rowstore enables support for all existing batch mode-enabled operators. Batch mode on Rowstore is a feature under the [**Intelligent Query Processing**](https://aka.ms/iqp) suite of features.  

This example will show you how upgrading to **Database Compatibility Level 150** could improve performance due to batch mode on rowstore, when your workload has the following characteristics:
* A significant part of the workload consists of analytical queries. Usually, these queries have operators like joins or aggregates that process hundreds of thousands of rows or more.
* The workload is CPU bound (If the bottleneck is I/O, we still recommend that you consider a columnstore index, if possible).
* If creating a columnstore index would add too much overhead to the transactional part of your workload; or if creating a columnstore index isn't feasible because your application depends on a feature that's not yet supported with columnstore indexes.

More information about this feature is available [here](https://docs.microsoft.com/sql/relational-databases/performance/intelligent-query-processing?view=sql-server-ver15#batch-mode-on-rowstore).

## Step 1: Setup WideWorldImportersDW database

You could choose to use a container to evaluate this feature. Create an instance of SQL Server 2019 using a Docker image and restore the WideWorldImportersDW database backup

You will need the **WideWorldImportersDW** database for this exercise. If you don't have this sample database, then you download the sample database [here](https://github.com/Microsoft/sql-server-samples/releases/download/wide-world-importers-v1.0/WideWorldImportersDW-Full.bak "WideWorldImportersDW-Full download").

Restore the copied WideWorldImportersDW database backup into the container and restore the backup.

##### Docker Commands
```
docker pull mcr.microsoft.com/mssql/server:2019-latest

docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=`<A Strong Password`>" -p 1445:1433 --name sql2019demo -d mcr.microsoft.com/mssql/server:2019-latest

docker cp ".\Downloads\WideWorldImportersDW-Full.bak" sql2019demo:/var/opt/mssql/data
```

**Note**: *For Linux installations the default path to use is /var/opt/mssql*


In [7]:
USE [master]
GO
IF EXISTS (SELECT [database_id] FROM sys.databases WHERE [name] = 'WideWorldImportersDW')
ALTER DATABASE [WideWorldImportersDW] SET SINGLE_USER WITH ROLLBACK IMMEDIATE
GO

DECLARE @datafilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultDataPath') AS VARCHAR(4000)) + 'WideWorldImportersDW.mdf'
DECLARE @logfilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultLogPath') AS VARCHAR(4000)) + 'WideWorldImportersDW.ldf'
DECLARE @inmemfilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultDataPath') AS VARCHAR(4000)) + 'WideWorldImportersDW_InMemory_Data_1'
DECLARE @secondaryfilepath VARCHAR(8000) = CAST(SERVERPROPERTY('InstanceDefaultDataPath') AS VARCHAR(4000))+ 'WideWorldImportersDW_2.ndf'

-- Change @backupfile file path as needed
DECLARE @backupfile VARCHAR(8000) = 'E:\SampleDBs\WideWorldImportersDW-Full.bak'
RESTORE DATABASE WideWorldImportersDW
FROM DISK = @backupfile 
WITH MOVE 'WWI_Primary' TO @datafilepath,
    MOVE 'WWI_UserData' TO @secondaryfilepath,
    MOVE 'WWIDW_InMemory_Data_1' TO @inmemfilepath,
    MOVE 'WWI_Log' TO @logfilepath, NOUNLOAD, REPLACE, STATS = 10
GO

USE [master]
GO
ALTER DATABASE [WideWorldImportersDW] MODIFY FILE ( NAME = N'WWI_Log', SIZE = 4GB )
GO

## Step 2: Enlarge the WideWorldImportersDW database

In [3]:
USE WideWorldImportersDW;
GO

/*
Assumes a fresh restore of WideWorldImportersDW
*/

IF OBJECT_ID('Fact.OrderHistory') IS NULL 
BEGIN
    SELECT [Order Key], [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
    INTO Fact.OrderHistory
    FROM Fact.[Order];
END;

ALTER TABLE Fact.OrderHistory
ADD CONSTRAINT PK_Fact_OrderHistory PRIMARY KEY NONCLUSTERED ([Order Key] ASC, [Order Date Key] ASC) WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_Stock_Item_Key
ON Fact.OrderHistory ([Stock Item Key])
INCLUDE(Quantity)
WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_OrderHistory_Quantity
ON Fact.OrderHistory ([Quantity])
INCLUDE([Order Key])
WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_OrderHistory_CustomerKey
ON Fact.OrderHistory([Customer Key])
INCLUDE ([Total Including Tax])
WITH (DATA_COMPRESSION = PAGE);
GO

IF (SELECT COUNT(*) FROM [Fact].[OrderHistory]) < 3702592
BEGIN
	DECLARE @i smallint
	SET @i = 0
	WHILE @i < 4
	BEGIN
		INSERT INTO [Fact].[OrderHistory] ([City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key])
		SELECT [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
		FROM [Fact].[OrderHistory];

		SET @i = @i +1
	END;
END
GO

IF OBJECT_ID('Fact.OrderHistoryExtended') IS NULL 
BEGIN
    SELECT [Order Key], [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
    INTO Fact.OrderHistoryExtended
    FROM Fact.[OrderHistory];
END;

ALTER TABLE Fact.OrderHistoryExtended
ADD CONSTRAINT PK_Fact_OrderHistoryExtended PRIMARY KEY NONCLUSTERED ([Order Key] ASC, [Order Date Key] ASC)
WITH (DATA_COMPRESSION = PAGE);
GO

CREATE INDEX IX_Stock_Item_Key
ON Fact.OrderHistoryExtended ([Stock Item Key])
INCLUDE (Quantity);
GO

IF (SELECT COUNT(*) FROM [Fact].[OrderHistory]) < 29620736
BEGIN
	DECLARE @i smallint
	SET @i = 0
	WHILE @i < 3
	BEGIN
		INSERT Fact.OrderHistoryExtended([City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key])
		SELECT [City Key], [Customer Key], [Stock Item Key], [Order Date Key], [Picked Date Key], [Salesperson Key], [Picker Key], [WWI Order ID], [WWI Backorder ID], Description, Package, Quantity, [Unit Price], [Tax Rate], [Total Excluding Tax], [Tax Amount], [Total Including Tax], [Lineage Key]
		FROM Fact.OrderHistoryExtended;

		SET @i = @i +1
	END;
END
GO

UPDATE Fact.OrderHistoryExtended
SET [WWI Order ID] = [Order Key];
GO

-- Repeat the following until log shrinks. These demos don't require much log space.
CHECKPOINT
GO
DBCC SHRINKFILE (N'WWI_Log' , 0, TRUNCATEONLY)
GO
SELECT * FROM sys.dm_db_log_space_usage
GO

## Step 3: Execute the query without Batch Mode on Rowstore

Even when the database compatibility level is set to the defauult (150), this can be done by using the USE HINT **DISALLOW_BATCH_MODE** query hint to disable the feature.


In [2]:
USE [WideWorldImportersDW];
GO
ALTER DATABASE [WideWorldImportersDW] SET COMPATIBILITY_LEVEL = 150;
GO
ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE;
GO

-- Row mode due to hint
SELECT	[Tax Rate],
		[Lineage Key],
		[Salesperson Key],
	SUM([Quantity])					AS SUM_QTY,
	SUM([Unit Price])				AS SUM_BASE_PRICE,
	COUNT(*)					AS COUNT_ORDER
FROM	[Fact].[OrderHistoryExtended]
WHERE	[Order Date Key]	<= dateadd(dd, -73, '2015-11-13')
GROUP	BY	[Tax Rate],
		[Lineage Key],
		[Salesperson Key]
ORDER	BY	[Tax Rate],
		[Lineage Key],
		[Salesperson Key]
OPTION (RECOMPILE, USE HINT('DISALLOW_BATCH_MODE'));
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(100 rows affected)

Total execution time: 00:00:04.930

Tax Rate,Lineage Key,Salesperson Key,SUM_QTY,SUM_BASE_PRICE,COUNT_ORDER
15.000,9,4,433280,243367.68,9216
15.000,9,6,836480,543988.48,18432
15.000,9,7,724480,846118.40,24576
15.000,9,8,862080,1028780.80,22144
15.000,9,9,1101056,924917.76,28928
15.000,9,11,2279296,3886913.28,62848
15.000,9,12,3262592,3852878.08,85888
15.000,9,15,5484800,6343098.88,144256
15.000,9,19,91831040,107590420.48,2310528
15.000,9,21,1240960,2060285.44,37632


Observe the query execution plan (or actual plan).

![BMOR_Disabled_Plan](./media/BMOR_disabled_plan.PNG)

Notice the time spent in each operator (cumulative up the tree for a query in row mode). Also note the two Aggregate operators. The Hash Match only does a partial aggregate, so later in the execxution a Stream Aggregate is needed to fully aggregate the result set as intended. Stream Aggregates are only possible in Row Mode.

Confirm the *Actual Execution Mode* was indeed "Row", for example in the properties of the *Table Scan* operator.

![BMOR_TableScan_properties](./media/BMOR_disabled_TableScan_properties.PNG)

## Step 4: Execute the query by removing any hint restriction

Run the same query from Step 3, but now without any hint, allowing SQL Server to operate as default. If a query is eligible for **Batch Mode on Rowstore**, it will be used instead of **Row Mode**. 

In [3]:
ALTER DATABASE [WideWorldImportersDW] SET COMPATIBILITY_LEVEL = 150;
GO

ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE;
GO

USE [WideWorldImportersDW]
GO
-- Batch mode on rowstore eligible
SELECT	[Tax Rate],
		[Lineage Key],
		[Salesperson Key],
	SUM([Quantity])					AS SUM_QTY,
	SUM([Unit Price])				AS SUM_BASE_PRICE,
	COUNT(*)					AS COUNT_ORDER
FROM	[Fact].[OrderHistoryExtended]
WHERE	[Order Date Key]	<= dateadd(dd, -73, '2015-11-13')
GROUP	BY	[Tax Rate],
		[Lineage Key],
		[Salesperson Key]
ORDER	BY	[Tax Rate],
		[Lineage Key],
		[Salesperson Key]
OPTION (RECOMPILE);

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(100 rows affected)

Total execution time: 00:00:01.818

Tax Rate,Lineage Key,Salesperson Key,SUM_QTY,SUM_BASE_PRICE,COUNT_ORDER
15.000,9,4,433280,243367.68,9216
15.000,9,6,836480,543988.48,18432
15.000,9,7,724480,846118.40,24576
15.000,9,8,862080,1028780.80,22144
15.000,9,9,1101056,924917.76,28928
15.000,9,11,2279296,3886913.28,62848
15.000,9,12,3262592,3852878.08,85888
15.000,9,15,5484800,6343098.88,144256
15.000,9,19,91831040,107590420.48,2310528
15.000,9,21,1240960,2060285.44,37632


Observe the query execution plan (or actual plan).

![BMOR_Enabled_Plan](./media/BMOR_enabled_plan.PNG)

Notice the time spent in each operator (not cumulative up the tree for a query in batch mode). Note the Stream Aggregate operator is not present. Also note that only one aggregate was required for the Batch Mode plan, which also improves the query execution.

Confirm the *Actual Execution Mode* was indeed "Batch", for example in the properties of the *Table Scan* operator.

![BMOR_enabled_TableScan_properties](./media/BMOR_enabled_TableScan_properties.PNG)

As you can see from the execution times, the query with Batch Mode for Rowstore finished much faster! From **~5s** to **~1.8s**.